# Path to file

In [1]:
import os
os.chdir("/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master")
!pwd
#/content/drive/MyDrive/contextualized-topic-models-master/contextualized_topic_models
#/content/drive/MyDrive/contextualized-topic-models-master-20240717T121550Z-001/contextualized-topic-models-master
!ls

/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master
AUTHORS.rst		     README.rst
contextualized_topic_models  readthedocs.yml
CONTRIBUTING.rst	     requirements_dev.txt
docs			     requirements.txt
HISTORY.rst		     setup.cfg
img			     setup.py
LICENSE			     tests
Makefile		     training_dataset_embeddings_summary.pkl
MANIFEST.in


# New Section

In [3]:
import pandas as pd
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing, WhiteSpacePreprocessingStopwords
import nltk

### Dataset Loading (Some of preprocessig steps are already been applied )

In [4]:
df = pd.read_csv("/home/sania/maktaba/topic_identification/segmentation/Summary.csv",encoding='utf-8')
# documents = df['text_ar'].values.tolist()
# len(set(df['label'].values.tolist()))
# home/sania/maktaba/topic_identification/segmentation/Original_Without_Stop_Stem.csv
df.head()

,Context,Summary,label
0,فيديو .. ثلاثون عاماً على كارثة ملعب هيسل\n أح...,عاشر عاماً على كارثة ملعب هيسل في بروكسل، حيث ...,Sport
1,كفيتوفا ترافق إيراني إلى رابع أدوار رولان غارو...,تأهلت بترا كفيتوفا التشيكية وساره إيراني الإيط...,Sport
2,أرسنال بطلا لكأس الاتحاد الإنكليزي للمرة الـ 1...,أحرز أرسنال لقب كأس الاتحاد الإنكليزي لكرة الق...,Sport
3,فولفسبورغ ينتزع كأس ألمانيا من بوروسيا دورتمو...,فولفسبورغ فاز بلقب كأس ألمانيا لكرة القدم بعد ...,Sport
4,عمالقة التنس يواصلون الصراع على لقب رولان غارو...,أصعد رافائيل نادال ونوفاك دجوكوفيتس أندي موراي...,Sport


In [5]:
documents = df['Summary'].values.tolist()
documents = [i for i in documents if type(i)!=float ]
len(documents)

4998

In [6]:
label_counts = df['label'].value_counts()

# Display the unique counts
print(label_counts)

label
Sport         556
Diverse       556
Economy       556
Politic       556
Technology    556
Religion      555
Art           555
Culture       554
Medical       554
Name: count, dtype: int64


In [7]:
documents[:5], len(documents)

(['عاشر عاماً على كارثة ملعب هيسل في بروكسل، حيث قتل شخص واحد وإصيب الكثيرين نتيجة تدافع كبير بين مشجعي ليفربول ويوفنتوس قبل المباراة النهائية لبطولة كأس الأندية الأوروبية. قام الرئيس الحالي للاتحاد الأوروبي لكرة القدم ميشيل بلاتيني بالذكرى هذه الكارثة والتحذير من تكرارها. الكارثة أدت إلى حرمان الأندية الإنكليزية من المشاركة في المسابقات الأوربية لفترة طويلة.',
  'تأهلت بترا كفيتوفا التشيكية وساره إيراني الإيطالية إلى الدور الرابع لبطولة فرنسا المفتوحة بعد فوزيهما على منافستيهما. سوف يلعب بترا ضد تيميا باشينسكي السويسرية، بينما سوف يلعب ساره ضد جوليا جورجيس الألمانية.',
  'أحرز أرسنال لقب كأس الاتحاد الإنكليزي لكرة القدم للمرة الـ 12 في تاريخه، بعد فوزه على أستون فيلا في المباراة النهائية. سجل أربعة أهداف في المباراة، كل من ثيو والكوت وأليكسيس سانشيز وبير ميرتيساكر وأوليفيه جيرو. هذا اللقب هو السادس الذي يفوز به أرسنال منذ تولي أرسين فينغر تدريب الفريق.',
  'فولفسبورغ فاز بلقب كأس ألمانيا لكرة القدم بعد تغلبه على بوروسيا دورتموند 3-1 في النهائي. سجل لويس غوستافو والبلجيكي كيفين دي برن 

### Data Preprocessing(Stopwords)

In [6]:
from nltk.corpus import stopwords as stop_words

nltk.download('stopwords')
stopwords = list(stop_words.words("arabic"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

[nltk_data] Downloading package stopwords to /home/sania/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
preprocessed_documents[:2]

['عاما ملعب قتل شخص نتيجة كبير المباراة النهايية لبطولة كاس الاوروبية الرييس الحالي الاوروبي لكرة القدم ميشيل بلاتيني الى المشاركة لفترة طويلة',
 'ايراني الى الدور الرابع لبطولة فرنسا المفتوحة يلعب ضد بينما يلعب ضد الالمانية']

### Embedding Creation (You can change embeddings to any model as you want depands on your demnad)

In [ ]:
import pickle

# Prepare the dataset with TopicModelDataPreparation
tp = TopicModelDataPreparation("sentence-transformers/LaBSE")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

# Save the dataset to a file
with open("training_dataset_embeddings_summary.pkl", "wb") as file:
    pickle.dump(training_dataset, file)

print("Embeddings saved to 'training_dataset_embeddings_summary.pkl'")


/home/sania/maktaba/topic_identification/segmentation/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models/utils/data_preparation.py:64: UserWarning: the longest document in your collection has 2645 words, the model instead truncates to 128 tokens.
  warnings.warn(
Batches: 100%|██████████| 25/25 [03:56<00:00,  9.47s/it]

Embeddings saved to 'training_dataset_embeddings_summary.pkl'


In [8]:
import pickle

# Load the embeddings from the file
with open("training_dataset_embeddings_summary.pkl", "rb") as file:
    training_dataset = pickle.load(file)

print("Embeddings loaded from 'training_dataset_embeddings.pkl'")


Embeddings loaded from 'training_dataset_embeddings.pkl'


In [9]:
training_dataset

In [10]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=7,dropout=0.2, num_epochs=80, learn_priors=False, lr=2e-3 )
ctm.fit(training_dataset) # run the model

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

In [11]:
ctm.get_topic_lists(8)

[['الله', 'الاسلام', 'وسلم', 'او', 'صلى', 'قال', 'والقيم', 'القران'],
 ['الشرطة', 'داعش', 'قتل', 'الامارات', 'ابوظبي', 'الشعر', 'الشارقة', 'مدينة'],
 ['بانها', 'تتميز', 'سجل', 'الروسي', 'الاتحاد', 'ضد', 'روسيا', 'القدم'],
 ['دراسة', 'تناول', 'اظهرت', 'الاصابة', 'اكثر', 'العلماء', 'يودي', 'ان'],
 ['النفط', 'اسعار', 'بنسبة', 'روسيا', 'برميل', 'مليون', 'تراجع', 'الامريكية'],
 ['بطولة',
  'الثالثة',
  'فيلم',
  'الموسيقية',
  'الذهبية',
  'سيكون',
  'فازت',
  'الدور'],
 ['ناسا', 'علماء', 'الفضاء', 'اكتشف', 'فضايية', 'باستخدام', 'كوكب', 'عثر']]

In [12]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5)

  0%|          | 0/79 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true

# Testing case (here f31.csv contains testing dataset)

In [29]:
import pandas as pd

# Load the test file
df2 = pd.read_csv("/home/sania/maktaba/topic_identification/segmentation/f31.csv")

# Extract English, Arabic texts, and their labels
document_english = df2['Arabic'].dropna().tolist()  # English text
document_arabic = df2['English'].dropna().tolist()    # Arabic text
actual_labels = df2['Tobic'].dropna().tolist()       # Corresponding labels

# Ensure all lists are the same length by trimming to the shortest
min_length = min(len(document_english), len(document_arabic), len(actual_labels))
document_english = document_english[:min_length]
document_arabic = document_arabic[:min_length]
actual_labels = actual_labels[:min_length]


In [30]:
# Transform the English and Arabic texts separately
testing_dataset_english = tp.transform(document_english)
testing_dataset_arabic = tp.transform(document_arabic)


/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models/utils/data_preparation.py:192: UserWarning: The method did not have in input the text_for_bow parameter. This IS EXPECTED if you are using ZeroShotTM in a cross-lingual setting
  warnings.warn(
/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models/utils/data_preparation.py:64: UserWarning: the longest document in your collection has 897 words, the model instead truncates to 128 tokens.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:03<00:00,  3.73s/it]
/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models/utils/data_preparation.py:192: UserWarning: The method did not have in input the text_for_bow parameter. This IS EXPECTED if you are using ZeroShotTM in a cross-lingual setting
  warnings.warn(
/home/sania/maktaba/topic_identification/segment

In [31]:
# Generate topic predictions for both languages
english_topics_predictions = ctm.get_thetas(testing_dataset_english, n_samples=8)
arabic_topics_predictions = ctm.get_thetas(testing_dataset_arabic, n_samples=8)


  0%|          | 0/2 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true 

In [32]:
import numpy as np
# Slice both lists to be the same length
min_length = min(len(document_arabic), len(document_english))
document_arabic = document_arabic[:min_length]
document_english = document_english[:min_length]

# Pair the Arabic and English samples together
testing_data = []
for i in range(min_length):
    testing_data.append(document_arabic[i])
    testing_data.append(document_english[i])

# Transform the data using the tp transformer
testing_dataset = tp.transform(testing_data)

/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models/utils/data_preparation.py:192: UserWarning: The method did not have in input the text_for_bow parameter. This IS EXPECTED if you are using ZeroShotTM in a cross-lingual setting
  warnings.warn(
/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models/utils/data_preparation.py:64: UserWarning: the longest document in your collection has 1600 words, the model instead truncates to 128 tokens.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:07<00:00,  7.25s/it]


In [33]:
# Define topic labels based on model topics
topic_labels = {
    0: "Sports",#Medical 
    1: "Medical",#Technology
    2: "Culture:",#Sports
    3: "Politics,technology",#Culture
    4: "Politics",#Economy
    5: "Religion",#Religion
    6: "Econmics/",#Politics
    # Add or adjust labels as necessary
}

Anaylsing if prediction and model generation is same or different

In [34]:
print("Persian Text Predictions")
for index in range(len(document_english)):
    # Get the actual label for English text
    actual_label = actual_labels[index]

    # Get the prediction scores for English text and sort them
    predictions = english_topics_predictions[index]
    sorted_predictions = [(idx, predictions[i]) for idx, i in enumerate(range(len(predictions)))]
    sorted_predictions = sorted(sorted_predictions, key=lambda x: x[1], reverse=True)

    # Identify the top predicted topic and map to a label
    topic_number = np.argmax(predictions)  # Index of the highest prediction score
    predicted_label = topic_labels.get(topic_number, "Unknown")

    # Display results for each English text
    print(f"Index: {index}")
    print(f"Text (English): {document_english[index][:100]}...")  # Show first 100 characters
    print(f"Actual Label: {actual_label}")
    print(f"Predicted Topic Index: {topic_number}")
    print(f"Predicted Label: {predicted_label}")
    print("Top Topics and Scores:", sorted_predictions)
    display(ctm.get_topic_lists(9)[topic_number])
    print(20 * "***")


Persian Text Predictions
Index: 0
Text (English): ينظم معهد الشارقة للفنون معرضاً فنياً تحت عنوان باقة الفن، وذلك عند الساعة السابعة من مساء اليوم في ...
Actual Label: Culture
Predicted Topic Index: 2
Predicted Label: Culture:
Top Topics and Scores: [(2, np.float32(0.73070055)), (1, np.float32(0.10556969)), (4, np.float32(0.064659745)), (0, np.float32(0.03171732)), (6, np.float32(0.027008949)), (3, np.float32(0.023050284)), (5, np.float32(0.017293379))]


['مهرجان',
 'تونس',
 'الفيلم',
 'المعرض',
 'فيلم',
 'الدورة',
 'الفنان',
 'مسلسل',
 'الشارقة']

************************************************************
Index: 1
Text (English): تقيم الفنانة ليتا كابيلوت معرضاً في مطلع العام المقبل في دبي، ويضم المعرض 35 لوحة، تركز خلالها على ا...
Actual Label: Culture
Predicted Topic Index: 2
Predicted Label: Culture:
Top Topics and Scores: [(2, np.float32(0.59086645)), (0, np.float32(0.11116871)), (6, np.float32(0.08563823)), (3, np.float32(0.076010495)), (5, np.float32(0.05757551)), (4, np.float32(0.048121236)), (1, np.float32(0.030619372))]


['مهرجان',
 'تونس',
 'الفيلم',
 'المعرض',
 'فيلم',
 'الدورة',
 'الفنان',
 'مسلسل',
 'الشارقة']

************************************************************
Index: 2
Text (English): تتواصل الليلة مسيرة التحدّي والمتعة والإثارة ضمن المنافسات على بيرق ولقب شاعر المليون في النسخة الثا...
Actual Label: Culture
Predicted Topic Index: 2
Predicted Label: Culture:
Top Topics and Scores: [(2, np.float32(0.35070652)), (3, np.float32(0.13140522)), (6, np.float32(0.12702116)), (1, np.float32(0.10667059)), (0, np.float32(0.10081869)), (5, np.float32(0.09854732)), (4, np.float32(0.0848305))]


...

************************************************************
Index: 73
Text (English):  رسول  صلى   سلم  صدق يهد  بر  بر يهد  جنة  رجل ليصدق  يكتب   صديقا  كذب يهد  فجور  فجور يهد  نار  ر...
Actual Label: Religion
Predicted Topic Index: 5
Predicted Label: Religion
Top Topics and Scores: [(5, np.float32(0.72469395)), (4, np.float32(0.06752559)), (1, np.float32(0.057220135)), (2, np.float32(0.047572654)), (0, np.float32(0.042037275)), (6, np.float32(0.041029226)), (3, np.float32(0.01992119))]


['الله', 'الاسلام', 'او', 'صلى', 'وسلم', 'المراة', 'الا', 'وكان', 'الاسلامية']

************************************************************
Index: 74
Text (English): يقيم  طليقته طلقن زوج يقيم  انا اولاده  بيت  بدعوى رعا اولاد فهل  جائز شرعاخ  مسقط يقول شيخ جمال قطب...
Actual Label: Religion
Predicted Topic Index: 5
Predicted Label: Religion
Top Topics and Scores: [(5, np.float32(0.5377792)), (6, np.float32(0.15521)), (3, np.float32(0.09761312)), (4, np.float32(0.089064755)), (0, np.float32(0.050780438)), (1, np.float32(0.0367269)), (2, np.float32(0.032825604))]


['الله', 'الاسلام', 'او', 'صلى', 'وسلم', 'المراة', 'الا', 'وكان', 'الاسلامية']

************************************************************


In [ ]:
print("FRENCH Text Predictions")
for index in range(len(document_arabic)):
    # Get the actual label for Arabic text
    actual_label = actual_labels[index]

    # Get the prediction scores for Arabic text and sort them
    predictions = arabic_topics_predictions[index]
    sorted_predictions = [(idx, predictions[i]) for idx, i in enumerate(range(len(predictions)))]
    sorted_predictions = sorted(sorted_predictions, key=lambda x: x[1], reverse=True)

    # Identify the top predicted topic and map to a label
    topic_number = np.argmax(predictions)  # Index of the highest prediction score
    predicted_label = topic_labels.get(topic_number, "Unknown")

    # Display results for each Arabic text
    print(f"Index: {index}")
    print(f"Text (Arabic): {document_arabic[index][:100]}...")  # Show first 100 characters
    print(f"Actual Label: {actual_label}")
    print(f"Predicted Topic Index: {topic_number}")
    print(f"Predicted Label: {predicted_label}")
    print("Top Topics and Scores:", sorted_predictions)
    display(ctm.get_topic_lists(9)[topic_number])
    print(20 * "***")


FRENCH Text Predictions
Index: 0
Text (Arabic): L'Institut d'art de Sharjah organise une exposition d'art intitulée « The Art Package » ce soir à 19...
Actual Label: Culture
Predicted Topic Index: 3
Predicted Label: Technology
Top Topics and Scores: [(3, np.float32(0.32754663)), (2, np.float32(0.18057725)), (0, np.float32(0.16711809)), (5, np.float32(0.16527271)), (6, np.float32(0.08077843)), (1, np.float32(0.044584483)), (4, np.float32(0.034122467))]


['شريط',
 'اخبارنا',
 'مشاهدة',
 'لديهم',
 'مشيرة',
 'شهير',
 'طويل',
 'احيانا',
 'جميل']

************************************************************
Index: 1
Text (Arabic): L'artiste Lita Capelot organisera une exposition au début de l'année prochaine à Dubaï. L'exposition...
Actual Label: Culture
Predicted Topic Index: 2
Predicted Label: Culture
Top Topics and Scores: [(2, np.float32(0.24776098)), (3, np.float32(0.16342485)), (0, np.float32(0.15693733)), (1, np.float32(0.1313159)), (5, np.float32(0.12336616)), (6, np.float32(0.11213066)), (4, np.float32(0.065064095))]


['فن', 'عرب', 'مسرح', 'ثقاف', 'مغرب', 'كتاب', 'مهرج', 'ثقافة', 'محمد']

************************************************************
Index: 2
Text (Arabic): Ce soir, le cortège de défis, de plaisir et d'excitation se poursuit dans le cadre du concours pour ...
Actual Label: Culture
Predicted Topic Index: 2
Predicted Label: Culture
Top Topics and Scores: [(2, np.float32(0.5222969)), (0, np.float32(0.18916708)), (6, np.float32(0.09877135)), (1, np.float32(0.080397904)), (5, np.float32(0.040268075)), (3, np.float32(0.03733602)), (4, np.float32(0.031762704))]


...

************************************************************
Index: 73
Text (Arabic): Un Messager, que les prières et la paix de Dieu soient sur lui, est véridique et détruit la justice....
Actual Label: Religion
Predicted Topic Index: 0
Predicted Label: Religion
Top Topics and Scores: [(0, np.float32(0.9393124)), (5, np.float32(0.017526638)), (6, np.float32(0.015498051)), (1, np.float32(0.008673564)), (4, np.float32(0.007502036)), (3, np.float32(0.0066680573)), (2, np.float32(0.0048192986))]


['اسلام', 'سلم', 'امر', 'صلى', 'تعالى', 'زوج', 'حق', 'مسلم', 'علماء']

************************************************************
Index: 74
Text (Arabic): Il réside avec son ex-femme, il a divorcé d'un mari qui réside avec ses enfants dans une maison au m...
Actual Label: Religion
Predicted Topic Index: 0
Predicted Label: Religion
Top Topics and Scores: [(0, np.float32(0.67026556)), (1, np.float32(0.12508129)), (5, np.float32(0.06847441)), (2, np.float32(0.039993808)), (6, np.float32(0.036713902)), (4, np.float32(0.03124097)), (3, np.float32(0.028230041))]


['اسلام', 'سلم', 'امر', 'صلى', 'تعالى', 'زوج', 'حق', 'مسلم', 'علماء']

************************************************************


# EVALUATION  METRTICES IN MODEL

In [14]:
import os
os.chdir("/home/sania/maktaba/topic_identification/segmentation/contextualized-topic-models-master/contextualized_topic_models")


In [4]:
! pip install --upgrade scipy

In [15]:
"""####Evaluate model"""

from contextualized_topic_models.evaluation.measures import CoherenceCV,CoherenceNPMI,TopicDiversity

texts = [doc.split() for doc in documents]

npmi_score = CoherenceNPMI(texts=texts, topics=ctm.get_topic_lists(10))
print(npmi_score.score())

cv_score = CoherenceCV(texts=texts, topics=ctm.get_topic_lists(10))
print(cv_score.score())

"""#####Extra topic details<br>
opic_details=ctm.get_topics(20)
"""

topic_details=pd.DataFrame(ctm.get_topic_lists(20))
topic_distr=ctm.get_topic_word_distribution()
doc_distr=ctm.get_doc_topic_distribution(training_dataset)
vocabulary=ctm.idx2token

ImportError: cannot import name 'triu' from 'scipy.linalg.special_matrices' (/home/sania/maktaba/topic_identification/segmentation/myenv/lib/python3.10/site-packages/scipy/linalg/special_matrices.py)

# Evaluation Metrices Precision , Recall , F1 

In [5]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

# Define the classes
classes = ['Culture', 'Medical', 'Sports', 'Technology', 'Religion', 'Politics', 'Economics']

# Create the confusion matrix
conf_matrix = np.array([
    [22,2, 6, 2, 2, 1,0	],  # Culture
    [0, 11,0, 1, 0, 0, 0],   # Medical
    [0, 0, 0, 0, 0, 0, 1],   # Sports
    [0, 0, 0, 1, 0, 0, 1],   # Technology
    [1, 0, 0, 0, 13, 0, 0],  # Religion
    [0, 0, 1, 1, 0, 0, 0],   # Politics
    [0, 1, 4, 0, 0, 0, 5]    # Economics
])

# Reconstruct y_true and y_pred from the confusion matrix
y_true = []
y_pred = []

for actual_class_idx, row in enumerate(conf_matrix):
    actual_class = classes[actual_class_idx]
    for predicted_class_idx, count in enumerate(row):
        predicted_class = classes[predicted_class_idx]
        y_true.extend([actual_class] * count)
        y_pred.extend([predicted_class] * count)

# Compute Overall Accuracy
overall_accuracy = accuracy_score(y_true, y_pred)

# Compute Per-Class Precision, Recall, F1-Score, and Support
precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, labels=classes, average=None, zero_division=0
)

# Compute Macro, Micro, and Weighted Averages
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = np.mean(f1)

micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=classes, average='micro', zero_division=0
)

weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=classes, average='weighted', zero_division=0
)

# Display Per-Class Metrics
print("Per-Class Metrics:\n")
print(f"{'Class':<12} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<7}")
for idx, cls in enumerate(classes):
    print(f"{cls:<12} {precision[idx]:<10.4f} {recall[idx]:<10.4f} {f1[idx]:<10.4f} {support[idx]:<7}")

# Display Average Metrics
print("\nAverage Metrics:\n")
print(f"{'Metric':<20} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
print(f"{'Macro Average':<20} {macro_precision:<10.4f} {macro_recall:<10.4f} {macro_f1:<10.4f}")
print(f"{'Micro Average':<20} {micro_precision:<10.4f} {micro_recall:<10.4f} {micro_f1:<10.4f}")
print(f"{'Weighted Average':<20} {weighted_precision:<10.4f} {weighted_recall:<10.4f} {weighted_f1:<10.4f}")

# Display Overall Accuracy
print(f"\nOverall Accuracy: {overall_accuracy:.4f} ({overall_accuracy * 100:.2f}%)\n")

# Optional: Comprehensive Classification Report
print("Detailed Classification Report:\n")
print(classification_report(y_true, y_pred, labels=classes, zero_division=0))


Per-Class Metrics:

Class        Precision  Recall     F1-Score   Support
Culture      0.9565     0.6286     0.7586     35     
Medical      0.7857     0.9167     0.8462     12     
Sports       0.0000     0.0000     0.0000     1      
Technology   0.2000     0.5000     0.2857     2      
Religion     0.8667     0.9286     0.8966     14     
Politics     0.0000     0.0000     0.0000     2      
Economics    0.7143     0.5000     0.5882     10     

Average Metrics:

Metric               Precision  Recall     F1-Score  
Macro Average        0.5033     0.4963     0.4822    
Micro Average        0.6842     0.6842     0.6842    
Weighted Average     0.8235     0.6842     0.7330    

Overall Accuracy: 0.6842 (68.42%)

Detailed Classification Report:

              precision    recall  f1-score   support

     Culture       0.96      0.63      0.76        35
     Medical       0.79      0.92      0.85        12
      Sports       0.00      0.00      0.00         1
  Technology       0.20    